# Analyzing CIA Factbook Data Using SQL

We are going to work through data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/). This is a compedium of statistics about all countries on Earth.The Factbook contains demographic information like the following:

* `population` — the global population.
* `population_growth` — the annual population growth rate, as a percentage.
* `area` — the total land and water area.

Installing `ipython-sql` into notebook

In [1]:
!conda install -yc conda-forge ipython-sql

Solving environment: ...working... done

# All requested packages already installed.



<h3> Connecting Jupyter Notebook to database file

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [3]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Database name is `facts`

# Exploring The Database

In [4]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* `name` — the name of the country.
* `area`— the country's total area (both land and water).
* `area_land` — the country's land area in square kilometers.
* `area_water` — the country's waterarea in square kilometers.
* `population` — the country's population.
* `population_growth`— the country's population growth as a percentage.
* `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
* `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

Conducting summary statistics to see what thet tell us

# Summary Statistics

In [5]:
%%sql 
SELECT MIN(population) AS min_pop, 
MAX(population) AS max_pop, 
MIN(population_growth) AS min_pop_growth, 
MAX(population_growth) AS max_pop_growth
    FROM facts;    

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


# Further Explore
It appears to show some weird results. According to the data there is:
* A country that as a population of 0
* A country that has a population of 7,256,490,011

These two numbers are rather small and big. Let us explore them further

<h3> Exploring MIN population

In [6]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MIN(population)
                            FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


<h3> Exploring MAX population

In [7]:
%%sql
SELECT *
    FROM facts
    WHERE population == (SELECT MAX(population)
                            FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


<h3> Exploring the bottom end of the list

In [8]:
%%sql
SELECT *
    FROM facts
    ORDER BY id DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None
260,oo,Southern Ocean,None,None,None,None,None,None,None,None
259,zn,Pacific Ocean,None,None,None,None,None,None,None,None
258,xo,Indian Ocean,None,None,None,None,None,None,None,None
257,zh,Atlantic Ocean,None,None,None,None,None,None,None,None
256,xq,Arctic Ocean,None,None,None,None,None,None,None,None
255,wi,Western Sahara,266000,266000,0,570866,2.82,30.24,8.34,None
254,we,West Bank,5860,5640,220,2785366,1.95,22.99,3.5,0.0
253,pg,Spratly Islands,5,5,0,None,None,None,None,None
252,pf,Paracel Islands,7,7,0,None,None,None,None,None


It appears the last row of database `facts` is a representation of the world. We will exclude this for future analysis

<h3> Validating Antarctica population </h3>
According to [Antarctica - The World Factbook](https://www.cia.gov/the-world-factbook/countries/antarctica/#people-and-society) on the CIA website, the population of Antarctica is indeed 0. However, there are research groups that periodically visit, but do not live there long term.
    